In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
mpl.rcParams["figure.dpi"] = 300
from skimage.io import imread

plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100  # 200 e.g. is really fine, but slower
plt.style.use("classic")
plt.style.use("seaborn-white")

plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100  # 200 e.g. is really fine, but slower

mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["ps.fonttype"] = 42
plt.rcParams["font.family"] = "Arial"

import re

import pandas as pd
import scanpy as sc

sc.set_figure_params(dpi=200)


import seaborn as sns
import skimage
from morphometrics.explore.cluster import cluster_features
from morphometrics.explore.dimensionality_reduction import pca
from morphometrics.utils.anndata_utils import table_to_anndata
from skimage.measure import label
from tqdm import tqdm

rng = np.random.default_rng(42)
import scipy.spatial.distance as distance

cm = 1 / 2.54  # centimeters in inches

In [ ]:
# Color palette
palette = {"Matrigel": "#17ad97", "No Matrix": "#4d4d4d", "Agarose": "#98d9d1"}

In [ ]:
# Load data
measurement_data = sc.read_h5ad(
    "h5_repro/actin_morphometrics_all_panel_defg_12_07_2024.h5ad"
)
# Create DF for Day, leiden, position
histo_time = pd.DataFrame()
histo_time["Day"] = np.array(measurement_data.obs["Day"]).astype(int)
histo_time["leiden"] = np.array(measurement_data.obs["leiden"]).astype(int)
histo_time["position"] = np.array(measurement_data.obs["position"]).astype(str)
histo_time["organoid"] = np.array(measurement_data.obs["organoid"]).astype(str)
histo_time["leiden"] = histo_time["leiden"].astype(str)
histo_time_2 = histo_time
measurement_data.obs["condition"] = measurement_data.obs["position"].astype(str)
measurement_data.obs.loc[
    measurement_data.obs["condition"] == "External ECM", "condition"
] = "Matrigel"
measurement_data.obs.loc[
    measurement_data.obs["condition"] == "Internal ECM", "condition"
] = "No Matrix"
measurement_data.obs.loc[
    measurement_data.obs["condition"] == "External diffusion barrier", "condition"
] = "Agarose"

In [ ]:
import seaborn as sns

sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(3, 2))
sns.despine(left=True, bottom=True, right=True)
measurement_data.obs["Axis length ratio"] = measurement_data.obs[
    "Axis_length_ratio_raw"
]
sns.boxplot(
    x="Day",
    y="Axis length ratio",
    data=measurement_data.obs,
    palette=palette,
    hue="condition",
    showfliers=False,
).legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=16)
plt.savefig("figures/axis_length_ratio_boxplot.pdf", bbox_inches="tight")

In [ ]:
# Calculate shannon
days = []
all_shannons = []
for day in np.unique(histo_time["Day"]):
    histo_time_day = histo_time[histo_time["Day"] == day]

    perturbations = []
    for perturbation in np.unique(histo_time["position"]):
        histo_time_perturb = histo_time_day[histo_time_day["position"] == perturbation]
        histo_time_perturbed_grouped = histo_time_perturb.groupby(["leiden"]).count()
        # print(histo_time_perturb)

        n = histo_time_perturbed_grouped["position"].sum()
        shannon_index = []
        for cluster in np.unique(histo_time["leiden"]):
            if (
                len(
                    histo_time_perturbed_grouped[
                        histo_time_perturbed_grouped.index == cluster
                    ]
                )
                != 0
            ):
                ni = histo_time_perturbed_grouped[
                    histo_time_perturbed_grouped.index == cluster
                ]["position"].iloc[0]
                pi = ni / n
                shannon_index.append(pi * np.log2(pi))
            else:
                shannon_index.append(0)
        shannon_index = np.array(shannon_index)
        shannon_index = -shannon_index.sum()
        perturbations.append(shannon_index)
        all_shannons.append([perturbation, day, shannon_index])
    days.append(perturbations)
all_shannons = pd.DataFrame(all_shannons, columns=["perturbation", "day", "Shannon"])
all_shannons[r"$\Delta$ Shannon index"] = (
    all_shannons["Shannon"] - all_shannons[all_shannons["day"] == 4]["Shannon"].min()
)

In [ ]:
# Bootstrap conf. int. by 1000 bootstraps
shannon_boot_straped = pd.DataFrame()
for j in tqdm(range(1000)):
    histo_time = resample(histo_time_2, random_state=j)
    days = []
    boot_strap_run = []
    for day in np.unique(histo_time["Day"]):
        histo_time_day = histo_time[histo_time["Day"] == day]

        perturbations = []
        for perturbation in np.unique(histo_time["position"]):
            histo_time_perturb = histo_time_day[
                histo_time_day["position"] == perturbation
            ]
            histo_time_perturbed_grouped = histo_time_perturb.groupby(
                ["leiden"]
            ).count()
            # print(histo_time_perturb)

            n = histo_time_perturbed_grouped["position"].sum()
            shannon_index = []
            for cluster in np.unique(histo_time["leiden"]):
                if (
                    len(
                        histo_time_perturbed_grouped[
                            histo_time_perturbed_grouped.index == cluster
                        ]
                    )
                    != 0
                ):
                    ni = histo_time_perturbed_grouped[
                        histo_time_perturbed_grouped.index == cluster
                    ]["position"].iloc[0]
                    pi = ni / n
                    shannon_index.append(pi * np.log2(pi))
                else:
                    shannon_index.append(0)
            shannon_index = np.array(shannon_index)
            shannon_index = -shannon_index.sum()
            perturbations.append(shannon_index)
            boot_strap_run.append([perturbation, day, shannon_index])
        days.append(perturbations)
    boot_strap_run = pd.DataFrame(
        boot_strap_run, columns=["perturbation", "day", "Shannon"]
    )
    shannon_boot_straped = pd.concat(
        [shannon_boot_straped, boot_strap_run["Shannon"]], axis=1
    )

In [ ]:
all_shannons[r"97.5 Shannon index"] = np.percentile(shannon_boot_straped, 97.5, 1)
all_shannons[r"2.5 Shannon index"] = np.percentile(shannon_boot_straped, 2.5, 1)
all_shannons[r"2.5 $\Delta$ Shannon index"] = (
    all_shannons["2.5 Shannon index"]
    - all_shannons[all_shannons["day"] == 4]["Shannon"].min()
)
all_shannons[r"97.5 $\Delta$ Shannon index"] = (
    all_shannons["97.5 Shannon index"]
    - all_shannons[all_shannons["day"] == 4]["Shannon"].min()
)
all_shannons.loc[all_shannons["perturbation"] == "External ECM", "perturbation"] = (
    "Matrigel"
)
all_shannons.loc[all_shannons["perturbation"] == "Internal ECM", "perturbation"] = (
    "No Matrix"
)
all_shannons.loc[
    all_shannons["perturbation"] == "External diffusion barrier", "perturbation"
] = "Agarose"

In [ ]:
plt.style.use("classic")
plt.style.use("seaborn-white")
plt.style.use("tableau-colorblind10")

rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.family": "Arial",
    "font.size": 5,
}
plt.rcParams.update(rc)

cm = 1 / 2.54  # centimeters in inches
fig, ax = plt.subplots(figsize=(12 * cm, 6 * cm))
sns.despine(left=True, bottom=True, right=True)


for perturbation in ["Matrigel", "No Matrix", "Agarose"]:
    data = all_shannons[all_shannons["perturbation"] == perturbation]

    ax.plot(
        data["day"],
        data[r"$\Delta$ Shannon index"],
        color=palette[perturbation],
        linewidth=1,
        label=perturbation,
    )

    ax.fill_between(
        data["day"],
        data[r"2.5 $\Delta$ Shannon index"],
        data[r"97.5 $\Delta$ Shannon index"],
        color=palette[perturbation],
        alpha=0.3,
    )

ax.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
ax.set_xlabel("Day")
ax.set_ylabel(r"$\Delta$ Shannon index")

plt.tight_layout()
plt.savefig("figures/shannon_bootstrapped.pdf", bbox_inches="tight")
plt.show()